Architecture of a FHIR app:
![image](https://cdn-images-1.medium.com/max/1200/1*qRd-H-cPHAGTB993sNLy3Q.png)
(from https://blog.heliossoftware.com/fhir-architectural-patterns-ae828b13d40c)

## BMI Calculator - ANSWERS

**Use Case**
A nurse practitioner wants to calculate a patient's body mass index (BMI). 


**Define Functional requirements**  
Step 1 - Define required clinical data elements  
- Patient height
- Patient weight
    
  
Step 2 - Identify FHIR Resources to support use case data (http://hl7.org/fhir/resourcelist.html)
- Patient height: Observation resource http://hl7.org/fhir/observation.html
- Patient weight: Observation resource http://hl7.org/fhir/observation.html

Step 3 - Identify data elements within each FHIR Resource definition necessary to support use case data

There are several with minor differences in meaning. Here we will use:
- Patient height: 8302-2
- Patient weight: 29463-7

## Initialize the environment to run the application

In [1]:
# Install the Python FHIR client package in the current Jupyter kernel using pip
import sys
!{sys.executable} -m pip install fhirclient

In [2]:
# Import the client library that understands how to make FHIR calls and 
# interpret the results
#  Then print 'FHIR client library has been loaded'
from fhirclient import client
print('FHIR client library has been loaded')

FHIR client library has been loaded


## Find a patient with relevant variables

*Hint: Use the URL https://fhirtest.uhn.ca/baseDstu3/ [resource]?code=[loinc]*

Here we will use patient 1307586

In [3]:
patientId = "1307586"

## Load the data and get basic demographics

In [4]:
# Load the Patient from the database, and display demographics
#   Define the FHIR Endpoint. We define the name of this app (used in SMART 
#   calls), and the location of the server
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://fhirtest.uhn.ca/baseDstu3'
}

In [5]:
# Create an instance of the FHIR client that points to this FHIR server. We use 
# this to make the calls.
db = client.FHIRClient(settings=settings)

In [6]:
# Perform a GET (read) on the patient with the ID you identified earlier
import fhirclient.models.patient as p
patient = p.Patient.read(patientId, db.server)

## *Optional: get basic demographics*

In [7]:
# Patient name. 
# Show the name object so we can see what's in it
print(patient.name)

[<fhirclient.models.humanname.HumanName object at 0x10955feb8>, <fhirclient.models.humanname.HumanName object at 0x109d86080>]


The patient's name is a list of two `HumanName`s. We can find out about the structure of this data type on the FHIR website: https://www.hl7.org/fhir/datatypes.html#HumanName

In [27]:
# Show the contents of each object in the list
for name in patient.name:
    print(name.as_json())

{'family': 'Simonis280', 'given': ['Kelsey155'], 'prefix': ['Mrs.'], 'use': 'official'}
{'family': 'Huels583', 'given': ['Kelsey155'], 'prefix': ['Mrs.'], 'use': 'maiden'}


In [9]:
# We see that a Patient can have more than one name (i.e. Patient.name is a list)
# and each name can have more than one given name (i.e. given is also a list).
# Let's use the first name in the list and the first given name:
first_name = patient.name[0].given[0]
last_name = patient.name[0].family
print(first_name, last_name)

Kelsey155 Simonis280


In [10]:
# Patient's gender
gender = patient.gender
gender

'female'

In [11]:
# Define variable dob as the patient's date of birth
dob = patient.birthDate.date
print(dob)

1968-10-24


In [13]:
# We'd like to know the patient's age. The EHR contains the date of birth only, 
# so we have to calculate the age using the dob and today's date.
# For this we need the datetime library
import datetime

# Define and print today variable.
today = datetime.date.today()
print(today)

2019-04-22


In [14]:
# Calculate the patient's age using the relativedelta method of the dateutil 
# module
from dateutil.relativedelta import relativedelta

delta = relativedelta(today, dob)
age = delta.years
age

50

In [16]:
# Print patient name, gender, DOB, age, and today's date
print("Patient's Name(s) =", first_name, last_name)
print("Gender =", gender)
print("DOB =", dob)
print("Today's Date =", today)
print("Patient's age =", age)

Patient's Name(s) = Kelsey155 Simonis280
Gender = female
DOB = 1968-10-24
Today's Date = 2019-04-22
Patient's age = 50


## Query database for relevant information

In [17]:
# Query the database for weight and print it with units
#  Remember to find the LOINC for weight

# allows search of Observation resources
import fhirclient.models.observation as obs

# specify the search, setting the patient, the code we want and the sort order 
# (descending order by date)
search = obs.Observation.where(
    struct={'patient': "Patient/"+patientId, 'code': "29463-7", '_sort': '-date'})

# show the actual url that is generated
url = search.construct()
print('Query: ', url)

Query:  Observation?patient=Patient%2F1307586&code=29463-7&_sort=-date


In [19]:
# Run the query
weight_observations = search.perform_resources(db.server)
if weight_observations:
    # because we sorted by date, the most recent weight observation comes first
    latest_weight = weight_observations[0]
    print(latest_weight.as_json())

{'id': '1307717', 'meta': {'lastUpdated': '2019-02-06T19:19:52.586+00:00', 'versionId': '1'}, 'category': [{'coding': [{'code': 'vital-signs', 'display': 'vital-signs', 'system': 'http://hl7.org/fhir/observation-category'}]}], 'code': {'coding': [{'code': '29463-7', 'display': 'Body Weight', 'system': 'http://loinc.org'}], 'text': 'Body Weight'}, 'context': {'reference': 'Encounter/1307714'}, 'effectiveDateTime': '2018-10-25T20:28:46-04:00', 'issued': '2018-10-25T20:28:46.216-04:00', 'status': 'final', 'subject': {'reference': 'Patient/1307586'}, 'valueQuantity': {'code': 'kg', 'system': 'http://unitsofmeasure.org', 'unit': 'kg', 'value': 101.16477135353011}, 'resourceType': 'Observation'}


We can also look at the definition of the `Observation` class here: https://www.hl7.org/fhir/observation.html#resource.
There we can see that the actual value will be in the `valueQuantity` field, which is of type `Quantity` (https://www.hl7.org/fhir/datatypes.html#quantity), which has fields `value` and `unit` 

In [20]:
weight_value = latest_weight.valueQuantity.value
weight_unit = latest_weight.valueQuantity.unit
print(weight_value, weight_unit)

101.16477135353011 kg


In [21]:
# Query the database for height and print it with units
#  Remember to find the LOINC for height

# specify the new search, setting the patient, the code we want and the sort
# order (descending order by date)
search = obs.Observation.where(
    struct={'patient': "Patient/"+patientId, 'code': "8302-2", '_sort': '-date'})

# show the actual url that is generated
url = search.construct()
print('Query: ', url)

Query:  Observation?patient=Patient%2F1307586&code=8302-2&_sort=-date


In [22]:
# height observations have a structure that is the same as the weight observation
# structure
height_observations = search.perform_resources(db.server)
if height_observations:
    latest_height = height_observations[0]
    print(latest_height.as_json())

{'id': '1307715', 'meta': {'lastUpdated': '2019-02-06T19:19:52.586+00:00', 'versionId': '1'}, 'category': [{'coding': [{'code': 'vital-signs', 'display': 'vital-signs', 'system': 'http://hl7.org/fhir/observation-category'}]}], 'code': {'coding': [{'code': '8302-2', 'display': 'Body Height', 'system': 'http://loinc.org'}], 'text': 'Body Height'}, 'context': {'reference': 'Encounter/1307714'}, 'effectiveDateTime': '2018-10-25T20:28:46-04:00', 'issued': '2018-10-25T20:28:46.216-04:00', 'status': 'final', 'subject': {'reference': 'Patient/1307586'}, 'valueQuantity': {'code': 'cm', 'system': 'http://unitsofmeasure.org', 'unit': 'cm', 'value': 174.20730086205384}, 'resourceType': 'Observation'}


In [23]:
height_value = latest_height.valueQuantity.value
height_unit = latest_height.valueQuantity.unit
print(height_value, height_unit)

174.20730086205384 cm


## Calculate BMI

In [25]:
# We now have all the information we need to calculate BMI.
# Calculate the BMI to one decimal
bmi = weight_value/((height_value/100)**2)
bmi = round(bmi, 1)
bmi

33.3

In [26]:
# check BMI category
#    Hint: See https://www.nhlbi.nih.gov/health/educational/lose_wt/risk.htm
#    for categories
if bmi < 18.5:
    print("BMI indicates patient is underweight")
elif 18.5 <= bmi <= 24.9:
    print("BMI indicates patient is within normal weight range")
elif 25 <= bmi <= 29.9:
    print("BMI indicates patient is overweight")
else:
    print("BMI indicates patient is obese")

BMI indicates patient is obese
